## Intro

The purpose of today's class is to explore data using **interactive visualizations**. Interactivity is a key part of modern dataviz. It's a way to allow users of your visualizations get their own feel for the data ... to create richer visualization, where people who use your work can expose more of the data by exploring.


In [1]:
import pandas as pd
pd.set_option('display.float_format', '{:.7f}'.format)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.io import show

## Part 1: Video Lectures and Reading

Starting this week, we'll be playing around with *explanatory data visualization*. Roughly speaking this means using data visualization to communicate your results to others. Thus, there are new things to think about. 

Until today we have worked with static data visualization. However, exploratory data analysis means to be able to explore the multi-faceted nature of data and *interactive dataviz* is a handy tool to do it! It allows to play with the data: Toggle the view. Zoom. Drag. Show more details. All those things. Those are a key part of modern data visualization. 

The video below provides context about these points.

We start with the video and then read a bit from a scientific article about types of explanatory dataviz. (*The video is from an old version of the class that used D3, so just ignore those parts.*).

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/yHKYMGwefso/0.jpg)](https://www.youtube.com/watch?v=yHKYMGwefso)

> *Exercises*: Explanatory data visualization

> * What are the three key elements to keep in mind when you design an explanatory visualization?
><font face="Comic sans MS" size="3" color="blue"> (1) Start with the question, what is that we cant to communicate. (2) allow exploration. (by making the visualization interective we can make our audians more engaged) (3) you should know your readers. (technical people or non technical managers) different prople have different level of understandings.   


> * In the video I talk about (1) *overview first*,  (2) *zoom and filter*,  (3) *details on demand*. 
>   - Go online and find a visualization that follows these principles (don't use one from the video). 
>   - Explain how your video achieves (1)-(3). It might be useful to use screenshots to illustrate your explanation.
> * Explain in your own words: How is explanatory data analysis different from exploratory data analysis?

## Part 2: Interactive visualizations with Bokeh



To really master interactive visualizations, you will need to work with JavaScript, especially [D3](https://d3js.org). Given the limited time available for this class, we can't squeeze that in. But luckily Python has some pretty good options for interactive visualizations. You can find a range of different options [here](https://mode.com/blog/python-interactive-plot-libraries/).

Today, we'll explore [`Bokeh`](https://docs.bokeh.org/en/latest/), which provides lots of nice interactive funtionalities to Python. To work with Bokeh, we first set up our system:

1. If you haven't installed it yet please do so. You can simply follow [these steps](https://docs.bokeh.org/en/latest/docs/first_steps/installation.html)

2. To include Bokeh in your notebooks you can follow the [Bokeh: Using with Jupyter](https://docs.bokeh.org/en/latest/docs/user_guide/output/jupyter.html#jupyter) guide. Come back to this one when you need it

3. We aim to give you a gentle start with Bokeh and I am going to include more example code than usual in the follwing.
   * **HINT 1**: If you're not an experienced Python user, I recommend going to the [official user's guide](https://docs.bokeh.org/en/latest/docs/user_guide.html#userguide) and working through it. Start by clicking "Introduction" in the linked page. That page has a glossary, a section on output methods, stuff on settings, and interfaces that you can scroll through. The next page *Basic Plotting* where the action is. Spend some time working through that.
   * **HINT 2**: And by "working through it", I mean copy, paste, and run the code in your own notebook. 

Ok. Let's get started. First a general announcement on the data.

> **Announcement**
> * During this entire lecture, as always, we are going to work with the SF Crime Data. 
> * We will use data for the **period 2010-2017***.


Now, to get you in the mood here's a little gif to illustrate what the goal of this exercise is:

![Movie](https://github.com/suneman/socialdata2023/blob/main/files/week8_1.gif?raw=true)

If the gif isn't displaying on your system, you can download it [here](https://github.com/suneman/socialdata2023/blob/main/files/week8_1.gif) and display locally.

> ***Exercise***: Recreate the results from **Week 2** as an interactive visualisation (shown in the gif). To complete the exercise, follow the steps below to create your own version of the dataviz.

### Data prep

A key step is to set up the data right. So for this one, we'll be pretty strict about the steps. The workflow is

1. Take the data for the period of 2010-2017 and group it by hour-of-the-day.
2. We would like to be able to easily compare how the distribution of crimes differ from each other, not absolute numbers, so we will work on *normalized data*:
    * To normalise data for within a crime category you simply to devide the count for each hour by the total number of this crime type. (To give a concrete example in the `ASSAULT` category, take the number of assault-counts in 1st hour you should devide by the total number of assaults, then you devide number of assaults in 2nd hour by the total number of assaults and so on)
    *  Your life will be easiest if you organize your dataframe as shown in [this helpful screenshot](https://github.com/suneman/socialdata2023/blob/main/files/W6_Part2_data.png).

In [2]:
#ake the data for the period of 2010-2017 and group it by hour-of-the-day.
data_link = 'D:\DTU\SP 2023\Social Data Viz\Data'
df = pd.read_pickle(data_link+'\ordered_Police_Department_Incident_Reports__Historical.pickle')

In [6]:
#df[(df['Category']=='BURGLARY') & (df['Hour']==1)].count

In [4]:
df_grouped = df.groupby(['Hour', 'Category']).size().unstack('Category')
df_grouped.head()

Category,ASSAULT,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,LARCENY/THEFT,PROSTITUTION,ROBBERY,STOLEN PROPERTY,TRESPASS,VANDALISM,VEHICLE THEFT,WEAPON LAWS
Hour,,,,,,,,,,,,,,
0,8878,3386,517,705,3686,817,18568,1342,2908,534,550,6352,4695,1087
1,8074,2186,380,647,2242,774,11481,1041,2987,379,393,4451,3206,815
2,7258,2358,341,642,1817,656,7505,771,3129,317,436,4210,2541,712
3,3557,2412,216,286,1235,258,4688,540,1888,246,359,2819,1572,439
4,2216,2143,158,106,900,132,3042,464,1225,222,260,1879,1215,308


In [10]:
df_normal = df_grouped.apply(lambda x: x / x.sum(), axis=0)
df_normal.head()

Category,ASSAULT,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,LARCENY/THEFT,PROSTITUTION,ROBBERY,STOLEN PROPERTY,TRESPASS,VANDALISM,VEHICLE THEFT,WEAPON LAWS
Hour,,,,,,,,,,,,,,
0,0.0544135,0.0380573,0.0525513,0.1267986,0.0316797,0.0844357,0.0399858,0.0815657,0.0544396,0.0477084,0.0294417,0.0567482,0.0376326,0.0532922
1,0.0494858,0.0245698,0.0386257,0.1163669,0.0192691,0.0799917,0.0247241,0.0632711,0.0559185,0.0338604,0.0210374,0.0397649,0.0256975,0.0399569
2,0.0444845,0.0265030,0.0346615,0.1154676,0.0156164,0.0677966,0.0161619,0.0468608,0.0585769,0.0283213,0.0233392,0.0376118,0.0203673,0.0349071
3,0.0218010,0.0271100,0.0219557,0.0514388,0.0106143,0.0266639,0.0100955,0.0328208,0.0353446,0.0219780,0.0192174,0.0251847,0.0126003,0.0215228
4,0.0135819,0.0240865,0.0160602,0.0190647,0.0077351,0.0136420,0.0065509,0.0282015,0.0229328,0.0198338,0.0139179,0.0167868,0.0097388,0.0151003


1. First, let's convert our **Pandas Dataframe** to **Bokeh ColumnDataSource**: 

In [106]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.io import show
source = ColumnDataSource(df_normal)

In [107]:
hour = list(df_normal.index)
hour = [str(i) for i in hour]
p = figure(x_range = FactorRange(factors=hour))

In [108]:
bar ={} # to store vbars
### here we will do a for loop:
for indx,i in enumerate(focuscrimes):
    bar[i] = p.vbar(x='Hour',  top=i, source= source, 
                    ### we will create a vbar for each focuscrime
                    legend_label=i,muted_alpha=0.1,muted =False) 

In [109]:
p.legend.click_policy="mute"

In [110]:
show(p)

In [22]:
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.io import show
source = ColumnDataSource(df_normal)
## it is a standard way to convert your df to bokeh

2. We also need to create an empty figure (we will add our stuff here later on). Mini sub-exercise: Find the a guide how to define a figure in Bokeh online. Here is a little help:
  > ```python
  > p = figure(...., x_range = FactorRange(factors=hours), ...) 
  > #p is a standard way to call figures in Bokeh
  > #make sure to add x_range. In my case hours is a list on the form ['1', '2', '3' ... , '24']
  > #read up on the FactorRange in the guide
  > #do not forget to add other attributes to the figure, e.g. title, axis names and so on
  > ```

In [23]:
hour = list(df_normal.index)
hour = [str(i) for i in hour]
p = figure(x_range = FactorRange(factors=hour))

3. Now we are going to add the bars. In order to do so, we will use **vbar** (see the guide for help):
  > ```python
  > bar ={} # to store vbars
  > ### here we will do a for loop:
  > for indx,i in enumerate(focuscrimes):
  >     bar[i] = p.vbar(x='name_of_the_column_that_contain_hours',  top=i, source= src, 
  >                     ### we will create a vbar for each focuscrime
  >                     legend_label=i,  muted_alpha=..., muted = ....) 
  > #i stands for a column that we use, top=y; we are specifying that our numbers comes from column i
  > #read up what legend_label, muted and muted_alpha do... you can add more attributes (you HAVE TO)
  > ```

In [34]:
focuscrimes = list(set(['WEAPON LAWS', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'ROBBERY', 'BURGLARY', 'ASSAULT', 'DRUNKENNESS', 'DRUG/NARCOTIC', 'TRESPASS', 'LARCENY/THEFT', 'VANDALISM', 'VEHICLE THEFT', 'STOLEN PROPERTY', 'DISORDERLY CONDUCT']))
bar ={} # to store vbars
### here we will do a for loop:
for indx,i in enumerate(focuscrimes):
    bar[i] = p.vbar(x='Hour',  top=i, source= source, 
                    ### we will create a vbar for each focuscrime
                    legend_label=i, muted_alpha=0.2) 

In [35]:
p.legend.click_policy="mute"

In [36]:
show(p)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "name_of_the_column_that_contain_hours" [renderer: GlyphRenderer(id='1079', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "hour" (closest match: "Hour") [renderer: GlyphRenderer(id='2836', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "hour" (closest match: "Hour") [renderer: GlyphRenderer(id='2875', ...)]
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a missp




If you've followed these steps, your data should be ready! Take a moment to celebrate. We now follow the [Bokeh guide for categorical data](https://docs.bokeh.org/en/latest/docs/user_guide/basic/bars.html):



1. First, let's convert our **Pandas Dataframe** to **Bokeh ColumnDataSource**: 
  > ```python
  > source = ColumnDataSource(your_processed_dataframe)
  > ## it is a standard way to convert your df to bokeh
  > ```
2. We also need to create an empty figure (we will add our stuff here later on). Mini sub-exercise: Find the a guide how to define a figure in Bokeh online. Here is a little help:
  > ```python
  > p = figure(...., x_range = FactorRange(factors=hours), ...) 
  > #p is a standard way to call figures in Bokeh
  > #make sure to add x_range. In my case hours is a list on the form ['1', '2', '3' ... , '24']
  > #read up on the FactorRange in the guide
  > #do not forget to add other attributes to the figure, e.g. title, axis names and so on
  > ```
3. Now we are going to add the bars. In order to do so, we will use **vbar** (see the guide for help):
  > ```python
  > bar ={} # to store vbars
  > ### here we will do a for loop:
  > for indx,i in enumerate(focuscrimes):
  >     bar[i] = p.vbar(x='name_of_the_column_that_contain_hours',  top=i, source= src, 
  >                     ### we will create a vbar for each focuscrime
  >                     legend_label=i,  muted_alpha=..., muted = ....) 
  > #i stands for a column that we use, top=y; we are specifying that our numbers comes from column i
  > #read up what legend_label, muted and muted_alpha do... you can add more attributes (you HAVE TO)
  > ```
4. The last thing to do is to make your legend interactive and display the figure:
  > ```python
  > p.legend.click_policy="mute" ### assigns the click policy (you can try to use ''hide'
  > show(p) #displays your plot
  > ```
5. You will notice that the legend appears in the middle of the figure (and it occludes some of the data). In order to fix this look into [this guide](https://stackoverflow.com/questions/26254619/position-of-the-legend-in-a-bokeh-plot) as a start. Below are some code snippets that you can use to deal with this problem (but read the guide first):
  > ```python
  > items = [] ### for the custom legend // you need to figure out where to add it
  > items.append((i, [bar[i]])) ### figure where to add it
  > legend = Legend(items=..., location=.....) ## figure where to add it
  > p.add_layout(...., ...) ## figure where to add it
  > ### if you read the guide, it will make sense
  > ```

Now you should be able to recreate this amazing visualisation.


**EXTRA feature**: If you're interested in detailed instructions for more Bokeh visualizations for your final project, you can find more inspiration **[here](https://github.com/suneman/socialdata2021/blob/main/lectures/Week8_extra_bokeh.ipynb)**.

## Part 3: Narrative Dataviz

Let's finish up with some reading

*Reading*: [Narrative Visualization: Telling Stories with Data](http://vis.stanford.edu/files/2010-Narrative-InfoVis.pdf) by Edward Segel and Jeffrey Heer. We'll read section 1-3 today. (And the rest a bit later).

When you get to section 3 it's fun to open up the examples mentioned by the authors in a browser and explore them as you read the text. 

> *Exercise*: Answer a couple of questions about the paper.
> 
> * What is the *Oxford English Dictionary's* defintion of a narrative?
> * What is your favorite visualization among the examples in section 3? Explain why in a few words.